In [20]:
! ls code

Dockerfile  docker-compose.yml	orig-compose.yaml
Makefile    myapp.py		requirements.txt


In [22]:
! cat code/myapp.py


from flask import Flask
from redis import Redis

app = Flask(__name__)
redis = Redis(host='redis', port=6379)

@app.route('/')
def hello():
    count = redis.incr('hits')
    return 'Hello World! I have been seen {} times.\n'.format(count)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=True)




In [24]:
! cat code/requirements.txt

flask
redis


In [25]:
! cat code/Dockerfile

FROM python:3.4-alpine
ADD . /code
WORKDIR /code
RUN pip install -r requirements.txt
CMD ["python", "myapp.py"]




In [26]:
! cat code/Makefile

Image='tbutzer/stackdemo'

build:
	docker build --tag ${Image} .

publish: ## Send this image to docker hub!
	sudo docker login -u tbutzer
	sudo docker push ${Image}

gnufix:
	sudo apt install gnupg2 pass

test:
	for i in 1 2 3 4 5 6\ ; do\
		curl localhost:8000 ; \
	done

shell:
	docker run --rm -it \
		-v /data3/butzer:/data \
		--name rose ${Image} /bin/bash


In [27]:
! cat code/docker-compose.yml

version: '3'

services:
  web:
    image: tbutzer/stackdemo
    build: .
    ports:
      - "8000:8000"
  redis:
    image: redis:alpine
  tony:
    image: tbutzer/bash-tester


In [1]:
!  docker stack deploy --compose-file code/docker-compose.yml stackdemo

Ignoring unsupported options: build

Updating service stackdemo_web (id: 22m2djkh71if2k88cfcjlzjkl)
Updating service stackdemo_redis (id: 6rpaa2jdfz5vmrh7dy4zfg1nz)
Updating service stackdemo_tony (id: 70n38yu5ut34mt5krmmvs9lv5)


In [2]:
!  docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                        PORTS
6rpaa2jdfz5v        stackdemo_redis     replicated          1/1                 redis:alpine                 
70n38yu5ut34        stackdemo_tony      replicated          1/1                 tbutzer/bash-tester:latest   
22m2djkh71if        stackdemo_web       replicated          1/1                 tbutzer/stackdemo:latest     *:8000->8000/tcp


In [3]:
!  docker service ps stackdemo_tony

ID                  NAME                IMAGE                        NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
salkofc1kl4s        stackdemo_tony.1    tbutzer/bash-tester:latest   mhonk               Running             Running 29 minutes ago                       


In [4]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` curl web:8000

Hello World! I have been seen 66 times.


In [5]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` curl stackdemo_web:8000

Hello World! I have been seen 67 times.


In [6]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host stackdemo_web

stackdemo_web has address 10.0.2.5


In [7]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host redis

redis has address 10.0.2.7


In [8]:
! docker network ls

NETWORK ID          NAME                DRIVER              SCOPE
44fe69509e08        bridge              bridge              local
190a23a05c45        docker_gwbridge     bridge              local
eacb4483945d        host                host                local
mbsdzmpfah84        ingress             overlay             swarm
d64b3a51d13c        none                null                local
yfszhlko6p92        stackdemo_default   overlay             swarm


In [9]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host 10.0.2.7

7.2.0.10.in-addr.arpa domain name pointer ip-10-0-2-7.us-west-2.compute.internal.


# Try this in a terminal

1. redis-cli -h redis
2. ping


In [10]:
! echo docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` bash

docker exec -it d8b52c135ada bash


In [11]:
! docker service inspect stackdemo_redis

[
    {
        "ID": "6rpaa2jdfz5vmrh7dy4zfg1nz",
        "Version": {
            "Index": 83
        },
        "CreatedAt": "2020-04-08T23:26:23.698461624Z",
        "UpdatedAt": "2020-04-08T23:55:36.167968092Z",
        "Spec": {
            "Name": "stackdemo_redis",
            "Labels": {
                "com.docker.stack.image": "redis:alpine",
                "com.docker.stack.namespace": "stackdemo"
            },
            "TaskTemplate": {
                "ContainerSpec": {
                    "Image": "redis:alpine@sha256:511275ddf4c4582e296f6fca803df853a8ab2b733a704c44c498f0a6a90e5ba0",
                    "Labels": {
                        "com.docker.stack.namespace": "stackdemo"
                    },
                    "Privileges": {
                        "CredentialSpec": null,
                        "SELinuxContext": null
                    },
                    "StopGracePeriod": 10000000000,
                    "DNSConfig": {},
                    "Isol

In [12]:
! docker service inspect stackdemo_redis | egrep -i 'ip|addr'

                "Mode": "vip"
                "Mode": "vip"
                "Mode": "vip"
            "VirtualIPs": [
                    "Addr": "10.0.2.7/24"


In [13]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host redis

redis has address 10.0.2.7


In [14]:
! sudo netstat -anlp | grep 8000

tcp6       0      0 :::8000                 :::*                    LISTEN      5551/dockerd        


https://dev.to/realabbas/explain-redis-like-i-m-five-4anm


Redis is an open source (BSD licensed), 
- in-memory data structure store, 
- used as a database, cache and message broker. -
- It supports data structures such as 
    - strings, 
    - hashes, -
    - lists, 
    - sets, 
    sorted sets with range queries, bitmaps, hyperloglogs, geospatial indexes with radius queries and streams. Redis has built-in replication, Lua scripting, LRU eviction, transactions and different levels of on-disk persistence, and provides high availability via Redis Sentinel and automatic partitioning with Redis Cluster. 

In [15]:
! docker service  ps stackdemo_redis

ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
n05npm23x4k3        stackdemo_redis.1   redis:alpine        mhonk               Running             Running 29 minutes ago                       


In [16]:
! docker service ls | awk '{print $5, $6}'

IMAGE PORTS
redis:alpine 
tbutzer/bash-tester:latest 
tbutzer/stackdemo:latest *:8000->8000/tcp


In [17]:
! curl localhost:8000

Hello World! I have been seen 68 times.


In [18]:
%%bash

for i in {1..20} ; do
{
    curl localhost:8000 
} ;
done

Hello World! I have been seen 69 times.
Hello World! I have been seen 70 times.
Hello World! I have been seen 71 times.
Hello World! I have been seen 72 times.
Hello World! I have been seen 73 times.
Hello World! I have been seen 74 times.
Hello World! I have been seen 75 times.
Hello World! I have been seen 76 times.
Hello World! I have been seen 77 times.
Hello World! I have been seen 78 times.
Hello World! I have been seen 79 times.
Hello World! I have been seen 80 times.
Hello World! I have been seen 81 times.
Hello World! I have been seen 82 times.
Hello World! I have been seen 83 times.
Hello World! I have been seen 84 times.
Hello World! I have been seen 85 times.
Hello World! I have been seen 86 times.
Hello World! I have been seen 87 times.
Hello World! I have been seen 88 times.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0  13333      0 --:--:-- --:--:-- --:--:-- 20000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0  20000      0 --:--:-- --:--:-- --:--:-- 20000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0  20000      0 --:--:-- --:--:-- --:--:-- 20000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0  20000      0 --:--:-- --:--:-- --:--:-- 20000
  % Total    % Received % Xferd  Average Speed   Tim

In [19]:
%%bash

CONTAINER=`docker ps |grep bash-tester | awk '{print $1}'`

echo ${CONTAINER}

for i in {1..12} ; do
{
    docker exec ${CONTAINER} curl web:8000
} ;
done


d8b52c135ada
Hello World! I have been seen 89 times.
Hello World! I have been seen 90 times.
Hello World! I have been seen 91 times.
Hello World! I have been seen 92 times.
Hello World! I have been seen 93 times.
Hello World! I have been seen 94 times.
Hello World! I have been seen 95 times.
Hello World! I have been seen 96 times.
Hello World! I have been seen 97 times.
Hello World! I have been seen 98 times.
Hello World! I have been seen 99 times.
Hello World! I have been seen 100 times.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   4866      0 --:--:-- --:--:-- --:--:--  5000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   6087      0 --:--:-- --:--:-- --:--:--  6666
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   5983      0 --:--:-- --:--:-- --:--:--  6666
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   6042      0 --:--:-- --:--:-- --:--:--  6666
  % Total    % Received % Xferd  Average Speed   Tim